In [4]:
from dotenv import load_dotenv
from langchain_community.vectorstores import SupabaseVectorStore
from langchain_ollama import OllamaEmbeddings
from supabase import create_client
from os import getenv

load_dotenv()

True

In [12]:
# Создание клиента Supabase
supabase_client = create_client(
    supabase_url=getenv("SUPABASE_URL"), supabase_key=getenv("SUPABASE_KEY")
)

In [13]:
embeddings_nomic = OllamaEmbeddings(
    model="nomic-embed-text", base_url="http://localhost:11434"
)

embeddings_gemma = OllamaEmbeddings(
    model="embeddinggemma", base_url="http://localhost:11434"
)

embeddings_qwen = OllamaEmbeddings(
    model="dengcao/Qwen3-Embedding-0.6B:Q8_0", base_url="http://localhost:11434"
)

In [14]:
# Создание vector stores для каждой таблицы с соответствующими эмбеддингами
vs1 = SupabaseVectorStore(
    client=supabase_client,
    embedding=embeddings_nomic,
    table_name="schema_embeddings_nomic",
    query_name="match_documents_nomic",
)

vs2 = SupabaseVectorStore(
    client=supabase_client,
    embedding=embeddings_gemma,
    table_name="schema_embeddings_gemma",
    query_name="match_documents_gemma",
)

vs3 = SupabaseVectorStore(
    client=supabase_client,
    embedding=embeddings_qwen,
    table_name="schema_embeddings_qwen06b",
    query_name="match_documents_qwen",
)

In [38]:
vector_stores = {
    "nomic": vs1,
    "gemma": vs2,
    "qwen_06B": vs3,  # добавь свои таблицы сюда
}

query = "tables with informations of all customers and sellers"
k = 5

for name, vs in vector_stores.items():
    results = vs.similarity_search_with_relevance_scores(query, k=k)
    print(f"\n=== Результаты для таблицы {name} ===")
    for doc, score in results:
        print(f"Score: {score:.4f} | Content: {doc.metadata.get("table_name")}")


=== Результаты для таблицы nomic ===
Score: 0.7454 | Content: sellers
Score: 0.6946 | Content: order_reviews
Score: 0.6903 | Content: customers
Score: 0.6803 | Content: products
Score: 0.6756 | Content: order_payments

=== Результаты для таблицы gemma ===
Score: 0.4966 | Content: customers
Score: 0.4507 | Content: sellers
Score: 0.4336 | Content: category_name_translation
Score: 0.4009 | Content: products
Score: 0.3804 | Content: geolocation

=== Результаты для таблицы qwen_06B ===
Score: 0.5255 | Content: sellers
Score: 0.4105 | Content: orders
Score: 0.3913 | Content: customers
Score: 0.3902 | Content: order_items
Score: 0.3757 | Content: category_name_translation
